# Mapping Suburbs to SA2 Areas
#### Jeremy

##### Dataset Obtained from
https://www.abs.gov.au/AUSSTATS/abs@.nsf/DetailsPage/1270.0.55.001July%202016?OpenDocument  
https://discover.data.vic.gov.au/dataset/rental-report-quarterly-moving-annual-rents-by-suburb

## Reading SA2 Data

In [ ]:
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Load the shapefiles
suburbs = gpd.read_file('../data/mappings/SAL_2021_AUST_GDA2020_SHP/SAL_2021_AUST_GDA2020.shp')
sa2_areas = gpd.read_file('../data/mappings/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')

print(suburbs.columns)
print(sa2_areas.columns)

In [ ]:
# Display sample text data from the suburbs GeoDataFrame
print("Suburbs Data Sample:")
print(suburbs[['SAL_NAME21', 'STE_NAME21', 'AUS_NAME21']].head())

# Display sample text data from the SA2 areas GeoDataFrame
print("\nSA2 Areas Data Sample:")
print(sa2_areas[['SA2_NAME21', 'SA3_NAME21', 'SA4_NAME21', 'GCC_NAME21', 'STE_NAME21']].head())

In [ ]:
# Filter to include only Victoria
suburbs_victoria = suburbs[suburbs['STE_NAME21'] == 'Victoria']
sa2_victoria = sa2_areas[sa2_areas['STE_NAME21'] == 'Victoria']

In [ ]:
# Plot the suburbs
plt.figure(figsize=(10, 10))
ax = suburbs_victoria.plot(edgecolor='k', alpha=0.5)
ax.set_title('Suburbs of Australia')
plt.show()

# Plot the SA2 areas
plt.figure(figsize=(10, 10))
ax = sa2_victoria.plot(edgecolor='k', alpha=0.5)
ax.set_title('SA2 Areas of Australia')
plt.show()

## Reading Moving Annual Rent by Suburb

In [ ]:
# Read all sheets into a dictionary of DataFrames
dfs_rent = pd.read_excel('../data/landing/Moving annual rent by suburb - March quarter 2024.xlsx', sheet_name=None)

# Access a specific sheet by its name
df_rent = dfs_rent['All properties'].drop(columns=['Moving annual rent by suburb'])
df_flat1 = dfs_rent['1 bedroom flat'].drop(columns=['Moving annual rent by suburb'])
df_flat2 = dfs_rent['2 bedroom flat'].drop(columns=['Moving annual rent by suburb'])
df_flat3 = dfs_rent['3 bedroom flat'].drop(columns=['Moving annual rent by suburb'])
df_house2 = dfs_rent['2 bedroom house'].drop(columns=['Moving annual rent by suburb'])
df_house3 = dfs_rent['3 bedroom house'].drop(columns=['Moving annual rent by suburb'])
df_house4 = dfs_rent['4 bedroom house'].drop(columns=['Moving annual rent by suburb'])

# Display the first few rows of the dataframe
print(df_rent['Unnamed: 1'].head(10))
print(df_rent.head(10))

In [ ]:
# Create a date range from March 2000 to March 2024, but only for quarters (March, June, September, December)
quarterly_range = pd.date_range(start='2000-03-01', end='2024-03-01', freq='QS-MAR')

# Generate the column names using the quarterly range, alternating between 'COUNT' and 'MEDIAN'
new_column_names = ['Suburbs']  # First column remains 'Suburbs'
for date in quarterly_range:
    quarter_year = date.strftime('%B%Y').upper()  # Format: 'MARCH2000', 'JUNE2000', etc.
    new_column_names.append(f'{quarter_year}COUNT')
    new_column_names.append(f'{quarter_year}MEDIAN')

# Map the old column names ('Unnamed: 1', 'Unnamed: 2', ...) to the new ones
old_column_names = ['Unnamed: ' + str(i) for i in range(1, len(new_column_names)+ 1)]

# Apply the renaming
# Apply the renaming for all DataFrames
df_rent = df_rent.rename(columns=dict(zip(old_column_names, new_column_names)))
df_flat1 = df_flat1.rename(columns=dict(zip(old_column_names, new_column_names)))
df_flat2 = df_flat2.rename(columns=dict(zip(old_column_names, new_column_names)))
df_flat3 = df_flat3.rename(columns=dict(zip(old_column_names, new_column_names)))
df_house2 = df_house2.rename(columns=dict(zip(old_column_names, new_column_names)))
df_house3 = df_house3.rename(columns=dict(zip(old_column_names, new_column_names)))
df_house4 = df_house4.rename(columns=dict(zip(old_column_names, new_column_names)))

# Rename specific column in all DataFrames
df_rent.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_flat1.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_flat2.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_flat3.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_house2.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_house3.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)
df_house4.rename(columns={'Lease commenced in year ending ': 'MARCH2000COUNT'}, inplace=True)

# Drop the specified rows from all DataFrames
df_rent = df_rent.drop([0, 1])
df_flat1 = df_flat1.drop([0, 1])
df_flat2 = df_flat2.drop([0, 1])
df_flat3 = df_flat3.drop([0, 1])
df_house2 = df_house2.drop([0, 1])
df_house3 = df_house3.drop([0, 1])
df_house4 = df_house4.drop([0, 1])

# Print out the new column names to verify
print(df_rent.columns)
print(df_rent.head(10))

In [ ]:
df_rent.to_csv('../data/raw/annual_moving_rent/all_properties.csv', index=False)
df_flat1.to_csv('../data/raw/annual_moving_rent/1_bedroom_flat.csv', index=False)
df_flat2.to_csv('../data/raw/annual_moving_rent/2_bedroom_flat.csv', index=False)
df_flat3.to_csv('../data/raw/annual_moving_rent/3_bedroom_flat.csv', index=False)
df_house2.to_csv('../data/raw/annual_moving_rent/2_bedroom_house.csv', index=False)
df_house3.to_csv('../data/raw/annual_moving_rent/3_bedroom_house.csv', index=False)
df_house4.to_csv('../data/raw/annual_moving_rent/4_bedroom_house.csv', index=False)

sa2_victoria.to_file("../data/raw/SA2_VIC/sa2_areas_vic.shp", driver='ESRI Shapefile')

## Mapping to SA2 Areas using Levenshtein Distance

In [ ]:
import pandas as pd

# Step 1: Load datasets
df_rent = pd.read_csv('../data/raw/annual_moving_rent/all_properties.csv')
df_flat1 = pd.read_csv('../data/raw/annual_moving_rent/1_bedroom_flat.csv')
df_flat2 = pd.read_csv('../data/raw/annual_moving_rent/2_bedroom_flat.csv')
df_flat3 = pd.read_csv('../data/raw/annual_moving_rent/3_bedroom_flat.csv')
df_house2 = pd.read_csv('../data/raw/annual_moving_rent/2_bedroom_house.csv')
df_house3 = pd.read_csv('../data/raw/annual_moving_rent/3_bedroom_house.csv')
df_house4 = pd.read_csv('../data/raw/annual_moving_rent/4_bedroom_house.csv')

# Step 2: Keep only relevant columns for each DataFrame
relevant_columns = ['Suburbs', 'MARCH2024MEDIAN', 'MARCH2024COUNT']

df_rent = df_rent[relevant_columns]
df_flat1 = df_flat1[relevant_columns]
df_flat2 = df_flat2[relevant_columns]
df_flat3 = df_flat3[relevant_columns]
df_house2 = df_house2[relevant_columns]
df_house3 = df_house3[relevant_columns]
df_house4 = df_house4[relevant_columns]


In [ ]:
df_flat1.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_FLAT1',
    'MARCH2024COUNT': 'MARCH2024COUNT_FLAT1'
}, inplace=True)

df_flat2.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_FLAT2',
    'MARCH2024COUNT': 'MARCH2024COUNT_FLAT2'
}, inplace=True)

df_flat3.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_FLAT3',
    'MARCH2024COUNT': 'MARCH2024COUNT_FLAT3'
}, inplace=True)

df_house2.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_HOUSE2',
    'MARCH2024COUNT': 'MARCH2024COUNT_HOUSE2'
}, inplace=True)

df_house3.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_HOUSE3',
    'MARCH2024COUNT': 'MARCH2024COUNT_HOUSE3'
}, inplace=True)

df_house4.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_HOUSE4',
    'MARCH2024COUNT': 'MARCH2024COUNT_HOUSE4'
}, inplace=True)

# For the rent DataFrame (assuming it's meant to have no specific type suffix)
df_rent.rename(columns={
    'MARCH2024MEDIAN': 'MARCH2024MEDIAN_RENT',
    'MARCH2024COUNT': 'MARCH2024COUNT_RENT'
}, inplace=True)

In [ ]:
dataframes_to_merge = [df_flat1, df_flat2, df_flat3, df_house2, df_house3, df_house4]

# Use reduce to merge all DataFrames iteratively
from functools import reduce

# Start with the rent DataFrame
combined_df = df_rent

# Define a function for merging while handling potential duplicates
def merge_dfs(left, right):
    return pd.merge(left, right, on='Suburbs', how='outer')

# Merge all DataFrames using reduce
combined_df = reduce(merge_dfs, dataframes_to_merge, combined_df)

# Remove duplicates if necessary
combined_df.drop_duplicates(subset='Suburbs', inplace=True)

# Display the combined DataFrame
print("Combined DataFrame:")
print(combined_df.head())
print(f"Total Rows: {len(combined_df)}")

In [ ]:
from fuzzywuzzy import process
import pandas as pd
import geopandas as gpd
import os

gdf_sa2 = gpd.read_file("../data/mappings/GDA94/vic_localities.shp")

# Extract the list of SA2 names
sa2_names = gdf_sa2['LOC_NAME'].tolist()

# Function to find the closest match
def get_best_match(suburb_name, choices):
    if pd.isna(suburb_name):
        return None
    match, score = process.extractOne(suburb_name, choices)
    return match

# Convert 'Suburbs' column to string and apply fuzzy matching
combined_df['Suburbs'] = combined_df['Suburbs'].astype(str)
combined_df['Suburb_name'] = combined_df['Suburbs'].apply(lambda x: get_best_match(x, sa2_names))

# Merge with the SA2 shapefile
# Use 'Best_Match_SA2' to merge on the SA2 shapefile's SA2_NAME21
merged_combined_df = pd.merge(gdf_sa2, combined_df, left_on='LOC_NAME', right_on='Suburb_name', how='left')

# Drop the 'Best_Match_SA2' column if no longer needed
merged_combined_df.drop(columns=['Suburb_name'], inplace=True)

# Remove rows where suburbs did not match (NaN after merge)
merged_combined_df.dropna(subset=['Suburbs'], inplace=True)

In [ ]:
# Define the output directory and file name for the merged dataset
output_dir = "../data/curated/sa2_rent/"
os.makedirs(output_dir, exist_ok=True)

# Name the output CSV file
csv_output_path = os.path.join(output_dir, "merged_final.csv")

# Save the merged DataFrame to a CSV
merged_combined_df.to_csv(csv_output_path, index=False)
print(f"Saved merged DataFrame to {csv_output_path}")

# Output number of rows and sample of the merged DataFrame
print(f"Number of rows in merged combined DataFrame: {len(merged_combined_df)}")
print(merged_combined_df.head())